In [1]:
library(readr)
library(dplyr)
library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



In [2]:
### Reading in Data
fp <- '/pool001/mfzhao/'
prcp <- read_csv(str_c(fp, "ECAD/prcp.txt"), col_names = F)
tmax <- read_csv(str_c(fp, "ECAD/tmax.txt"), col_names = F)

Parsed with column specification:
cols(
  X1 = col_double(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_double(),
  X5 = col_double()
)

Parsed with column specification:
cols(
  X1 = col_double(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_double(),
  X5 = col_double()
)



In [3]:
colnames(prcp) <- c('STAID', 'SOUID', 'DATE', 'PRCP', 'Q_PRCP')
colnames(tmax) <- c('STAID', 'SOUID', 'DATE', 'TMAX', 'Q_TMAX')

prcp %>% 
    filter(DATE >= 20100101, Q_PRCP == 0) %>%
    select(-SOUID, -Q_PRCP) %>%
    full_join(tmax %>% 
              filter(DATE >= 20100101, Q_TMAX == 0) %>%
              select(-SOUID, -Q_TMAX)) -> weather

Joining, by = c("STAID", "DATE")



In [4]:
write_csv(weather, str_c(fp, 'PROCESSED_DATA/ecad_weather.csv'))

In [5]:
stations <- read_csv('/pool001/mfzhao/ECAD/stations.txt', skip = 17)

stations %>% 
    separate(LAT, sep = ':', into = c('lat1', 'lat2', 'lat3')) %>%
    separate(LON, sep = ':', into = c('lon1', 'lon2', 'lon3')) %>%
    mutate(lat1 = as.numeric(lat1),
           lat2 = as.numeric(lat2),
           lat3 = as.numeric(lat3),
           lon1 = as.numeric(lon1),
           lon2 = as.numeric(lon2),
           lon3 = as.numeric(lon3),
           lat = ifelse(lat1 >=0, lat1 + lat2/60 + lat3/3600, lat1 - lat2/60 - lat3/3600),
           lng = ifelse(lon1 >=0, lon1 + lon2/60 + lon3/3600, lon1 - lon2/60 - lon3/3600)) %>%
    select(STAID, STANAME, CN, lat, lng) -> stations

Parsed with column specification:
cols(
  STAID = col_double(),
  STANAME = col_character(),
  CN = col_character(),
  LAT = col_character(),
  LON = col_character(),
  HGHT = col_double()
)



In [6]:
write_csv(stations, str_c(fp, 'PROCESSED_DATA/ecad_stations.csv'))